### Problem Statement:
#### The dataset contains inconsistencies, missing values, formatting errors, and noise in multiple columns. The goal is to clean the data to ensure accuracy, consistency, and usability.

### Objective:
#### To apply data cleaning techniques to improve the quality of the dataset.

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
#loading dataset
df = pd.read_csv("messy_data.csv")

In [3]:
df.head()

,Unnamed: 0,ID,Name,Age,Email,Join Date,Salary,Department
0,0,1e407ff9-6255-489d-a0de-34135d4f74bd,Hunter Thomas,25.0,xlopez@hotmail.com,NaN,88552.0,Sales
1,1,379f55b8-87d5-4739-a146-7400b78c24d1,Jeremy Irwin,90.0,Jillian Jenkins,2022-07-07,139227.0,NaN
2,2,18261368-dfa1-47f0-afc6-bddf45926b07,Jennifer Hammondquickly,66.0,jscottgreen.biz,2023-11-21,65550.0,Engineering
3,3,ae7cf7cf-17cf-4c8b-9c44-4f61a9a238e5,Sydney Taylorso,39.0,luke56gonzalez.com,2021-11-05,139932.0,SupportJ
4,4,14ed3e6a-e0f5-4bbe-8d93-8665267f5c90,Julia Lee,71.0,figueroakayla@yahoo.com,NaN,143456.0,Marketing


In [4]:
#shape of the dataset
df.shape

(11000, 8)

In [5]:
#statistical summary
df.describe()

,Unnamed: 0,Age,Salary
count,11000.000000,9253.000000,8761.000000
mean,5012.947818,54.162650,89886.585012
std,2884.739158,21.072919,34896.320117
min,0.000000,18.000000,24655.136613
25%,2509.750000,36.000000,59723.844874
50%,5024.500000,54.000000,89241.000000
75%,7510.250000,72.000000,119491.000000
max,9999.000000,90.000000,176156.206747


In [6]:
# checking the null values
df.isnull().sum()

Unnamed: 0       0
ID               0
Name          2333
Age           1747
Email         1269
Join Date     2192
Salary        2239
Department    2255
dtype: int64

#### We can observe that null values exist in 6 columns. Lets handle missing values using appropriate imputation techniques.

In [7]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
df.head()

,ID,Name,Age,Email,Join Date,Salary,Department
0,1e407ff9-6255-489d-a0de-34135d4f74bd,Hunter Thomas,25.0,xlopez@hotmail.com,NaN,88552.0,Sales
1,379f55b8-87d5-4739-a146-7400b78c24d1,Jeremy Irwin,90.0,Jillian Jenkins,2022-07-07,139227.0,NaN
2,18261368-dfa1-47f0-afc6-bddf45926b07,Jennifer Hammondquickly,66.0,jscottgreen.biz,2023-11-21,65550.0,Engineering
3,ae7cf7cf-17cf-4c8b-9c44-4f61a9a238e5,Sydney Taylorso,39.0,luke56gonzalez.com,2021-11-05,139932.0,SupportJ
4,14ed3e6a-e0f5-4bbe-8d93-8665267f5c90,Julia Lee,71.0,figueroakayla@yahoo.com,NaN,143456.0,Marketing


In [9]:
print(df['Join Date'].dtype)

object


In [10]:
df['Join Date'] = pd.to_datetime(df['Join Date'], errors='coerce')

In [11]:
# filling missing name with "No name provided"
df["Name"].fillna("No name provided", inplace = True)

# impute missing values(Age) with median
df["Age"].fillna(df["Age"].median(), inplace = True)

# impute missing values(Salary) with median
df["Salary"].fillna(df["Salary"].median(), inplace = True)

# filling missing emails as "Unknown"
df["Email"].fillna("Unknown", inplace = True)

# fill missing dates with "Unknown"
df["Join Date"] = df["Join Date"].fillna(pd.Timestamp('2025-01-01'))

# filling missing department with "unknown"
df["Department"].fillna("Unknown", inplace = True)

print("Fill null values succesfully")

Fill null values succesfully


In [12]:
# duplicated value sum
df.duplicated().sum()

299

In [13]:
# to find the no:of duplicated rows
duplicate_rows = df[df.duplicated()]
print("No:of duplicate rows = ", len(duplicate_rows))

No:of duplicate rows =  299


In [14]:
# to remove duplicated rows
df.drop_duplicates(inplace=True)

In [15]:
# verify that duplicates have been removed
print("Duplicated row after removal = ", len(df[df.duplicated()]))

Duplicated row after removal =  0


In [16]:
# display remaining missing values after handling
df.isnull().sum()

ID            0
Name          0
Age           0
Email         0
Join Date     0
Salary        0
Department    0
dtype: int64

#### After handling missing values, dataset is now complete and ready for further analysis.

In [17]:
# validate email format
import re
df = df[df["Email"].str.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')]

df = df[~df['Email'].str.contains('gmail|yahoo|hotmail')]

print("Validate email format successfully")

Validate email format successfully


In [18]:
# clean name field
df["Name"] = df["Name"].str.replace(r'[^a-zA-Z\s]','',regex = True).str.strip()
print("Remove noise successfully")

Remove noise successfully


In [19]:
# standardize date format
df['Join Date'] = pd.to_datetime(df['Join Date'], errors='coerce').dt.strftime('%Y-%m-%d')
print("Standardize date format successfully")

Standardize date format successfully


In [20]:
# Define a reasonable salary range
MIN_SALARY = 10000   
MAX_SALARY = 500000  

# Remove salaries outside the defined range
df = df[(df['Salary'] >= MIN_SALARY) & (df['Salary'] <= MAX_SALARY)]

# Handling Outliers using IQR (Interquartile Range)
Q1 = df['Salary'].quantile(0.25)  # First quartile 
Q3 = df['Salary'].quantile(0.75)  # Third quartile 
IQR = Q3 - Q1  # Interquartile range

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df = df[(df['Salary'] >= lower_bound) & (df['Salary'] <= upper_bound)]

print("Salary noise handled successfully.")

Salary noise handled successfully.


In [21]:
# correct department names
df['Department'] = df['Department'].str.strip().str.capitalize()

from thefuzz import process 

# Define the correct department names
valid_departments = ["HR", "Engineering", "Marketing", "Sales", "Support"]

# Function to correct department names using fuzzy matching
def correct_department(dept):
    if pd.isna(dept) or dept.strip() == "":
        return "Unknown"  # Assign a default value for missing departments
    match, score = process.extractOne(dept, valid_departments)
    return match if score > 80 else "Unknown"  # Assign "Unknown" if confidence is low

# Apply correction to the Department column
df['Department'] = df['Department'].apply(correct_department)

print("Department names standardized successfully.")

Department names standardized successfully.


In [22]:
df.head()

,ID,Name,Age,Email,Join Date,Salary,Department
5,f92ea13f-24d3-4792-91e4-5b0e07d90c2c,Lisa Bailey,81.0,milleryvonne@fowler-york.com,2025-01-01,123018.000000,Support
9,d06f507d-62e5-4bd6-a252-4e60e109d324,Jay Fisher,44.0,denisemaynard@jordan.com,2022-12-08,37701.725993,HR
10,d5fe9321-49cf-4f14-ac96-7aea1204247b,Craig Conway,88.0,gerald47@woodard-smith.com,2025-01-01,81214.525972,Engineering
14,9b83a5e5-9516-493f-86f8-d6c445edb501,Nicole Cardenas,60.0,griffinlinda@harrison.com,2025-01-01,51625.814403,Marketing
15,0074dbd5-10d0-463f-9b10-d8473bea04ba,No name provided,22.0,ericjohnson@castillo-mendez.net,2023-03-01,89241.000000,HR


In [23]:
df.reset_index(drop=True, inplace=True)

In [24]:
df.head()

,ID,Name,Age,Email,Join Date,Salary,Department
0,f92ea13f-24d3-4792-91e4-5b0e07d90c2c,Lisa Bailey,81.0,milleryvonne@fowler-york.com,2025-01-01,123018.000000,Support
1,d06f507d-62e5-4bd6-a252-4e60e109d324,Jay Fisher,44.0,denisemaynard@jordan.com,2022-12-08,37701.725993,HR
2,d5fe9321-49cf-4f14-ac96-7aea1204247b,Craig Conway,88.0,gerald47@woodard-smith.com,2025-01-01,81214.525972,Engineering
3,9b83a5e5-9516-493f-86f8-d6c445edb501,Nicole Cardenas,60.0,griffinlinda@harrison.com,2025-01-01,51625.814403,Marketing
4,0074dbd5-10d0-463f-9b10-d8473bea04ba,No name provided,22.0,ericjohnson@castillo-mendez.net,2023-03-01,89241.000000,HR


In [25]:
df.shape

(3587, 7)

In [26]:
# Save the cleaned dataset
df.to_csv('cleaned_data.csv', index=False)

In [27]:
df.to_excel("cleaned_data.xlsx", index=False)